In [9]:
import os
import os.path as osp
import argparse
from cv2 import imread, normalize, resize
import cv2
import json
import numpy as np
import time
import datetime
import random
import logging
import sys
from PIL import Image
from pytorchtools import EarlyStopping # 위 링크의 깃허브 파일에서 임포트
from utils import euclidean_metric, one_hot, count_acc
from utils import pprint, set_gpu, ensure_path, AverageMeter, Timer, accuracy, one_hot

import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch import nn, optim, autograd
from torch.utils.data import DataLoader
import torch.utils.data as data
import torchvision
from random import sample, random

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset, Dataset, TensorDataset, WeightedRandomSampler
import os.path as osp
from tqdm import tqdm

from random import sample, random

import clip
import skimage
import IPython.display
import matplotlib.pyplot as plt

from collections import OrderedDict
import torch

batch_size = 32
num_epoch = 100

In [10]:
class Customdataset(data.Dataset):
    def __init__(self,transform = None):
        dalist=None
        self.file_paths = []
        self.label = []
        self.concept = []
        self.transform = transform

        folder_list = ["custom_blur","custom_brightness","custom_cameraz","custom_lightness"]
        
        for folder_name in folder_list:
            path_dir = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/augmented_image2/train1-1/" +  folder_name
            file_list = os.listdir(path_dir)
            
            for file_name in file_list:
                self.file_paths.append(path_dir + '/' + file_name)
                self.label.append(file_name.split('_')[-1][0])
                
                if str(file_name.split('_')[2]) == 'cameraz':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'blur':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'brigntness':
                    self.concept.append(2)
                    
                    
                elif str(file_name.split('_')[2]) == 'lightness':
                    self.concept.append(1)

        
        
        file_path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/train.1-1.txt' # 폴더 경로

        with open(file_path) as f:
            lines = f.readlines()

        image_names = [line.rstrip('\n') for line in lines]
        
                
        with open('/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as json_file:
            data = json.load(json_file)
            
        path_image = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/image/"
        files = os.listdir(path_image)
        
        for file in files:
            if file in image_names:
                self.label.append(data['single_image'][file]['class'][0])
                self.file_paths.append(path_image + '/' + file)
                self.concept.append(0)
        
        
                
        file_path = '/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/imageset/single_image.2class.subject-split.trainval-repeat0/fold.5-5/ratio/100%/validation.1-1.txt' # 폴더 경로

        for folder_name in folder_list:
            path_dir = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/augmented_image2/train1-1/" +  folder_name
            file_list = os.listdir(path_dir)
            
            for file_name in file_list:
                self.file_paths.append(path_dir + '/' + file_name)
                self.label.append(file_name.split('_')[-1][0])
                
                if str(file_name.split('_')[2]) == 'cameraz':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'blur':
                    self.concept.append(3)
                    
                elif str(file_name.split('_')[2]) == 'brigntness':
                    self.concept.append(2)
                    
                    
                elif str(file_name.split('_')[2]) == 'lightness':
                    self.concept.append(1)
        
        with open(file_path) as f:
            lines = f.readlines()

        image_names = [line.rstrip('\n') for line in lines]
        
                
        with open('/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/annotation/single_image.6class.json', 'r') as json_file:
            data = json.load(json_file)
            
        path_image = "/home/iai/Desktop/SH/cognex/d-sub-15pin_for-seoultech/image/"
        files = os.listdir(path_image)
        
        for file in files:
            if file in image_names:
                self.label.append(data['single_image'][file]['class'][0])
                self.file_paths.append(path_image + '/' + file)
                self.concept.append(0) 

    def __len__(self):

        return len(self.file_paths)

    def __getitem__(self, idx):
        
        label = self.label[idx]
        idx_num = idx
        tmp_prob = np.random.rand()
        weight_dict = {"0" :  (400/250) / (400/250 + 400 * 5 / 30), "1" : (400/30) / (400/250 + 400 * 5 / 30), "2" : (400/30) / (400/250 + 400 * 5 / 30), 
                       "3" : (400/30) / (400/250 + 400 * 5 / 30), "4" : (400/30) / (400/250 + 400 * 5 / 30), "5" : (400/30) / (400/250 + 400 * 5 / 30)}        
        
        if weight_dict[str(label)] > tmp_prob:
            while True : 
                idx_num = np.random.randint(len(self.file_paths))
                tmp_prob =  np.random.rand()
                label = self.label[idx_num]
                
                if weight_dict[str(label)] < tmp_prob:
                    break
        
        
        path = self.file_paths[idx_num]
        image = Image.open(path).convert('RGB')
        concept = self.concept[idx_num]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, int(label), int(concept)

In [11]:
data_transforms = transforms.Compose([
        
        transforms.Resize(size=224),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),

        ])

##커스텀 데이터셋
dataset = Customdataset(transform=data_transforms)
train_size = int(0.8 * len(dataset))
print(f"train size :{train_size}")
val_size = len(dataset) - train_size
print(f"test_size :{val_size}")
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f"train_dataset :{len(train_dataset)}")
print(f'test dataset :{len(val_dataset)}')


train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size = batch_size, #배치사이즈
                                               shuffle = True 
                                               )

val_loader = torch.utils.data.DataLoader(val_dataset,
                                               batch_size = batch_size,                                              
                                               shuffle = True 
                                              )

train size :8576
test_size :2144
train_dataset :8576
test dataset :2144


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
"""
model, preprocess = clip.load("RN50", device=device)
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)
"""

'\nmodel, preprocess = clip.load("RN50", device=device)\nmodel.cuda().eval()\ninput_resolution = model.visual.input_resolution\ncontext_length = model.context_length\nvocab_size = model.vocab_size\n\nprint("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")\nprint("Input resolution:", input_resolution)\nprint("Context length:", context_length)\nprint("Vocab size:", vocab_size)\n'

In [13]:
zero_shot_model, zero_shot_preprocess= clip.load("RN50", device=device)

# Freeze the model
for param in zero_shot_model.parameters():
    param.requires_grad = False
    
fine_tunning_model ,fine_tunning_preprocess= clip.load("RN50", device=device)

In [14]:
zero_shot_preprocess

Compose(
    Resize(size=224, interpolation=PIL.Image.BICUBIC)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7f89d0e26a60>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [15]:
import torch.distributions as distributions

def custom_loss(zero_shot_output_context, fine_output_context, fine_output_label, target, alpha):
    # Compute the cross entropy loss
    ce_loss = F.cross_entropy(fine_output_label, target)
    print(ce_loss)
    # Compute the KL divergence between the output and the target
    kl_loss = F.kl_div(zero_shot_output_context, fine_output_context)
    print(ce_loss)
    
    # Return the mean of the cross entropy and KL divergence losses
    return ce_loss + alpha * kl_loss

In [16]:
model_path = '/home/iai/Desktop/SH/cognex/model/'
optimizer = torch.optim.Adam(fine_tunning_model.parameters(), lr=0.00005)
#optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.05, momentum=0.9, nesterov=True, weight_decay=0.0005) 
#lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)


early_stopping = EarlyStopping(patience = 4, verbose = True, min_epoch=20)

global_count = 0
device = torch.device('cuda:0')

train_loss_arr =[]
val_loss_arr =[]
n = len(train_loader)

label_list =['ok', 'scratch', 'fm', 'pin', 'dent', 'glue']
context_list = ['repeat', 'lightness' , 'brigntness' , 'blur', 'cameraz']

label_text = []
for label in label_list:
    label_text.append("a photo of " + str(label))
    
context_label_text = []
for label in label_list:
    for context in context_list:
        context_label_text.append("a " +str(context) + " of " + str(label))

label_text = clip.tokenize(label_text).to(device)
context_label_text = clip.tokenize(context_label_text).to(device)


for epoch in range(1, num_epoch + 1):
    
    #fine_tunning_model.train()
    train_running_loss = 0.0
       
    for i,[image,label, context] in enumerate(train_loader):
        
        target = label.to(device)
        input_image = image.to(device)
        
        zero_shot_image_features = zero_shot_model.encode_image(input_image)
        zero_shot_context_text_features = zero_shot_model.encode_text(context_label_text)
        
        zero_shot_image_features /= zero_shot_image_features.norm(dim=-1, keepdim=True)
        zero_shot_context_text_features /= zero_shot_context_text_features.norm(dim=-1, keepdim=True)
        zero_shot_output_context = (zero_shot_image_features @ zero_shot_context_text_features.T).softmax(dim=-1)
        
        
        fine_tunning_image_features = fine_tunning_model.encode_image(input_image)
        fine_tunning_context_text_features = fine_tunning_model.encode_text(context_label_text)
        
        fine_tunning_image_features /= fine_tunning_image_features.norm(dim=-1, keepdim=True)
        fine_tunning_context_text_features /= fine_tunning_context_text_features.norm(dim=-1, keepdim=True)
        fine_tunning_output_context = (fine_tunning_image_features @ fine_tunning_context_text_features.T).softmax(dim=-1)
        
        
        fine_tunning_label_text_features = fine_tunning_model.encode_text(label_text)
        fine_tunning_label_text_features /= fine_tunning_label_text_features.norm(dim=-1, keepdim=True)
        fine_tunning_output_label = (fine_tunning_image_features @ fine_tunning_label_text_features.T).softmax(dim=-1)
        
        print(zero_shot_output_context.shape)
        print(fine_tunning_output_context.shape)
        print(fine_tunning_output_label.shape)
        print(target.shape)
        
       
        loss = custom_loss(zero_shot_output_context, fine_tunning_output_context, fine_tunning_output_label, target, alpha=1)
        #ce_loss = F.cross_entropy(fine_tunning_output_label, target)
        #kl_loss = F.kl_div(zero_shot_output_context, fine_tunning_output_context)
        #loss = ce_loss + 1 * kl_loss
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        torch.cuda.synchronize()
        train_running_loss += loss.item()
        
        
    train_loss_arr.append(train_running_loss / n)


torch.Size([32, 30])
torch.Size([32, 30])
torch.Size([32, 6])
torch.Size([32])
tensor(1.7920, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(1.7920, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.HalfTensor [6, 1024]], which is output 0 of MmBackward, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).